1. Imports

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter   import RecursiveCharacterTextSplitter
import streamlit as st
from langchain.vectorstores import FAISS

2. chunk the text created in 1_webcrawling

In [5]:
TEXT_PATH = "data/all_text.txt"   # the file you wrote earlier

# ─── 3)  load the whole file into one LangChain Document ─────────────────────────
loader = TextLoader(TEXT_PATH, encoding="utf-8")
docs   = loader.load()                  # → [Document(page_content=..., metadata={'source': TEXT_PATH})]

# ─── 4)  split into overlapping chunks ───────────────────────────────────────────
splitter = RecursiveCharacterTextSplitter(
    chunk_size     = 1000,   # characters per chunk
    chunk_overlap  = 200,    # to give the model context continuity
    separators     = ["\n\n", "\n", " ", ""],  # big → small
)
chunks = splitter.split_documents(docs)   # list[Document]

print(f"{len(chunks)=}")
print(chunks[0].page_content[:300] + " [...]")

len(chunks)=1203
# https://wiki.student.uni-goettingen.de/start [...]


3. create embeddings

In [6]:
from helper.academicCloudEmbeddings import AcademicCloudEmbeddings

embedder = AcademicCloudEmbeddings(api_key=st.secrets["GWDG_API_KEY"], url=st.secrets["BASE_URL_EMBEDDINGS"])
store = FAISS.from_documents(chunks, embedder)
store.save_local("faiss_wiki_index")      # → folder with index